<img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">

<h1 style="text-align:center;">CSCI 420/520: Introduction to Machine Learning, Fall 2017</h1>
<h1 style="text-align:center;">CNNs in Keras for the MNIST data set</h1>

#### Credit where credit is due

This notebook is based on code taken from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py.

# Contents

* [Getting set up](#Getting-set-up)
* [Prepare the data](#Prepare-the-data)
* [Specify the CNN](#Specify-the-CNN)
* [Examine the model](#Examine-the-model)
    * [Understanding the number of model parameters](#Understanding-the-number-of-model-parameters)
    * [The layers in the CNN](#The-layers-in-the-CNN)
* [Training the CNN](#Training-the-CNN)
    * [A single epoch](#A-single-epoch)
    * [Why is the training loss higher than the testing loss?](#Why-is-the-training-loss-higher-than-the-testing-loss?)
* [Looking at the softmax probability estimates](#Looking-at-the-softmax-probability-estimates)
* [Evaluating the model](#Evaluating-the-model)
* [More epochs](#More-epochs)
* [Keras callbacks](#Keras-callbacks)
* [Looking at the trained network](#Looking-at-the-trained-network)
* [Saving the model](#Saving-the-model)
* [Data augmentation](#Data-augmentation)
* [A textbook neural network](#A-textbook-neural-network)
    * [The model parameters](#The-model-parameters)
    * [Training the network](#Training-the-network)

# Getting set up

You will need to do the following to get Keras and TensorFlow working.

1. Revert to Python 3.5: <code>conda install python=3.5</code>
2. Install TensorFlow 1.4: <code>pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.4.0-py3-none-any.whl</code>
3. Specify TensorFlow as the backend for Keras: in <code>~/.keras/keras.json</code> change <code>theano</code> to <code>tensorflow</code>
4. Install Keras: <code>pip install keras</code>

# Prepare the data

Once again we will use the MNIST handwritten digit dataset.

In [ ]:
import numpy as np

def read_mnist (vile):
    """
    This function reads the MNIST .npy files and returns the feature vectors and their associated
    class labels, and a list of the class labels.
    """
    Z = np.load(vile)
    m, n = Z.shape
    X = np.float32(Z[:, 0:n-1])
    y = Z[:, n-1]
    classes = np.unique(y)
    return X, y, classes

print('Reading the data...', end='')
x_train, y_train, classes = read_mnist('mnist_train.npy')
x_test,  y_test,  classes = read_mnist('mnist_test.npy')
print('done!')

print('Shape of x_train after prefrobincation:', x_train.shape)
print('Number of training cases: {0:5d}'.format(x_train.shape[0]))
print('Number of test cases:     {0:5d}'.format(x_test.shape[0]))

## Scale the data

Since the gray scale values are 8-bit integers in the range 0 to 255, we'll just divide by 255.

In [ ]:
print('Scaling data to the range [0,1]...', end='')
x_train /= 255
x_test  /= 255
print('done!')

## Encode the class labels

We'll use one-hot encoding for the class labels.

In [ ]:
from keras import utils as kutils

# The number of classes.
num_classes = 10

print('Encoding the class labels...', end='')
y_train_enc = kutils.to_categorical(y_train, num_classes)
y_test_enc  = kutils.to_categorical(y_test,  num_classes)
print('done!')

## Perform some pre-flight checks

Make sure we understand the organization of the image data.

In [ ]:
from keras import backend as K

# The input image dimensions.
img_rows, img_cols = 28, 28

# The number of classes.
num_classes = 10

# Check whether the channels (e.g., RGB) should come first or second and indicate this in
# the tuple input_shape.  This isn't an issue for the grayscale MNIST data.
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test  = x_test.reshape(x_test.shape[0],   1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test  = x_test.reshape(x_test.shape[0],   img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('Shape of x_train after prefrobincation:', x_train.shape)
print('Number of training cases: {0:5d}'.format(x_train.shape[0]))
print('Number of test cases:     {0:5d}'.format(x_test.shape[0]))

# Specify the CNN

The layers of the CNN are as follows:
1. the input layer (not explicitly specified),
2. a convolutional layer consisting of 32 filters, each 3 x 3,
3. a convolutional layer consisting of 64 filters, each 3 x 3,
4. a pooling layer to downsample the output of layer 2,
5. a dropout layer which randomly selects nodes to turn off for dropout regularization,
6. a flattening layer to turn the 2d image into a 1d vector,
7. a standard layer with all-to-all connections,
8. a second dropout layer, and finally
9. a standard layer with all-to-all connections that produces as output softmax estimates of the class probabilities.

In addition, in the call to the call to the model's [<code>compile</code>](https://keras.io/models/sequential) method we must also specify the [optimization algorithm](https://keras.io/optimizers) to be used in training the model.

In [ ]:
import tensorflow as tf
from keras import models, layers, losses, optimizers

print('constructing the model...', end='')

model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),
          activation='relu',
          input_shape=input_shape))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

print('done!')

## The loss function: cross-entropy

The measure of goodness of the model (the **loss function**) is cross-entropy.

If there are $T$ training cases and $K$ classes, the cross-entropy is
$$
-\frac{1}{T} \sum_{t=1}^{T} \sum_{k=1}^{K} y_{k}^{(t)} \log p_{k}^{(t)},
$$
where
$$
y_{k}^{(t)} = \left\{
    \begin{array}{cl}
        1 & \mbox{if $k$ is the class of training case $t$}; \\
        0 & \mbox{otherwise}
    \end{array}
  \right.
$$
and
$$
p_{k}^{(t)} = \mbox{estimated probability that training case $t$ is in class $k$}.
$$
The goal is to **minimize** the cross-entropy.

To understand the cross-entropy, let case $t$ belong to class $k$, so $y_{k}^{(t)} = 1$.

Suppose the NN completely misclassifies case $t$: $p_{k}^{(t)} = 0$.  Then the cross-entropy contains the term
$$
1 \log 0 = -\infty,
$$
which makes the cross-entropy infinite (note the minus sign in the defintion above).  

On the other hand, if the NN is entirely certain case $t$ is of class $k$, then $p_{k}^{(t)} = 1$, and the contribution to the cross-entropy is 
$$
1 \log 1 = 0.
$$

# Examine the model

We can get a summary of the model using the <code>summary()</code> method.

In [ ]:
model.summary()

## Understanding the number of model parameters

Here is how to interpret the number of parameters.
- In the first layer, there are 32 units, each with its own 3x3 filter.  This makes 9 x 32 = 288 parameters.  In addition, each of the 32 units has a bias term, for a total of 288 + 32 = 320 model parameters.
- The second convolutional layers takes the 32 convolutions from the first layer and feeds them to 64 units.  This means there are 32 x 64 filters, each 3x3, for a total of 32 x 64 x 9 = 18432.  There are also 64 bias terms for the second layer, so we end up with 18432 + 64 = 18496 model parameters
- The dense layer is taking the 9216 outputs from the flattening layer and feeding them to a dense layer with 128 units.  This makes for 9216 x 128 = 1,179,648 model parameters.  Addding the 128 bias terms yields 1,179,776 model parameters.
- Finally, in the last layer, we have 128 outputs being fed to 10 units, each with a bias term, leading to 10 x 128 + 10 = 1290 model parameters.

This model has a large number of parameters to determine during the training phase.  The dropout regularization help address overfitting due to the large number of parameters.

## The layers in the CNN

In [ ]:
for layer in model.layers:
  print(layer)

We can also visualize the CNN as a graph using the [Keras plot_model utility](https://keras.io/utils/#plot_model).

This requires the following Python packages:
1. pydot,
2. graphviz.

In addition, you will need to install the [GraphViz](http://graphviz.org) graph visualization package.

In [ ]:
from IPython.display import Image

kutils.plot_model(model, to_file='model.png')

Image('model.png')

# Training the CNN

With large training sets one frequently takes optimization steps based on an approximate direction of steepest descent computed using only a subset of the the data.  We refer to the size of the subset used as the **batch size**.

An **epoch** is a pass through the entire data set in the process of training/optimization.

In each epoch, the batches are chosen randomly, whence the name **stochastic gradient descent** (SGD).

SGD allows us to try steps more quickly, and avoid the time and space requirements of processing the entire training set.  If the training data are reasonably uniform in their distribution, a subset of the data should give results similar to the entire training set.

## A single epoch

We will use a single epoch of training to start.  This is done with the method [<code>fit</code>](https://keras.io/models/sequential) in the <code>Sequential</code> model class.

We also will use the test data as validation data.  This is useful to detect overfitting.

In [ ]:
batch_size = 128
epochs = 5

model.fit(x_train, y_train_enc,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test_enc))

score = model.evaluate(x_test, y_test_enc, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

## Why is the training loss higher than the testing loss?

Notice that the accuracy reported for the training set is **lower** than that reported for the test set, and the loss report for the training set is **higher** than that reported for the test set.  This seems backwards.

Per the [Keras FAQ](https://keras.io/getting-started/faq/#why-is-the-training-loss-much-higher-than-the-testing-loss):

<blockquote>
<p>
A Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and L1/L2 weight regularization, are turned off at testing time.
</p>
<p>
Besides, the training loss is the average of the losses over each batch of training data. Because your model is changing over time, the loss over the first batches of an epoch is generally higher than over the last batches. On the other hand, the testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.
</p>
</blockquote>

# Looking at the softmax probability estimates

In Keras it is possible to [examine the outputs of the layers in the network](https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer).

Let's take a look at the softmax probabilities for the first 11 test cases, and compare with the actual labels.

The softmax layer is the eighth and last layer, which is layer 7 since we are counting from 0.

In [ ]:
get_softmax_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                      [model.layers[7].output])

num_cases = 11
softmax = get_softmax_layer_output([x_test[0:num_cases,:], 0])[0]

print('label  estimated probabilities by class')
print('       ', end='')
for c in range(0, num_classes):
    print('{0:3d}  '.format(c), end='')
print()
for t in range(0, num_cases):
    print('{0:<5d}  '.format(y_test[t]), end='')
    for c in range(0, num_classes):
        print('{0:.1f}  '.format(softmax[t][c]), end='')
    print()

In [ ]:
H = np.load('hillary.npy')
H = H.reshape(H.shape[0], img_rows, img_cols, 1)

In [ ]:
 get_softmax_layer_output([H, 0])[0]

# Evaluating the model

For this model the method <code>predict()</code> returns the softmax probability estimates for each image.  We will take as our classification the class with the maximum estimated probability.

As we have in the past we will look at the confusion matrix, precision, and recall for our model.

In [ ]:
from sklearn import metrics

def compute_metrics (classifier, X_test, y_test, classes):
    """
    This function computes and prints various performance measures for a classifier.
    """
    # Use the classifier to make predictions for the test set.
    y_pred = classifier.predict(X_test)
    
    # Choose the class with the highest estimated probability.
    y_pred = np.argmax(y_pred, axis=1)

    print('Classes:', classes, '\n')

    # Compute the confusion matrix.
    cm = metrics.confusion_matrix(y_test, y_pred, labels=classes)
    print('Confusion matrix, without normalization')
    print(cm, '\n')

    # Normalize the confusion matrix by row (i.e by the number of samples in each class).
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    np.set_printoptions(precision=3, linewidth=132)
    print('Normalized confusion matrix')
    print(cm_normalized, '\n')

    # The confusion matrix as percentages.
    cm_percentage = 100 * cm_normalized
    print('Confusion matrix as percentages')
    print(np.array2string(cm_percentage, formatter={'float_kind':lambda x: "%6.2f" % x}), '\n')
    
    # Precision, recall, and f-score.
    print(metrics.classification_report(y_test, y_pred, digits=3))

    return cm

compute_metrics(model, x_test, y_test, classes)

# More epochs 

Keras makes it easy to continue the training where we left off earlier.  It saves the information needed to continue training the neural network.

When we specify <code>initial_epoch</code>, the value of <code>epochs</code> is the epoch at which training ends, rather than the number of epochs we train over.

That is, if we set <code>initial_epoch=3</code> and <code>epochs=4</code>, then we will only optimize over a single epoch.

This convention is useful if we want to keep track of how many epochs of training have been performed.

In [ ]:
batch_size = 128
epochs = 2

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          initial_epoch = 1)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

# Keras callbacks 

Keras has a number of [callback](https://keras.io/callbacks) functions that can be used to specify actions to occur during training.

Callbacks include
* <code>ModelCheckpoint</code>, which saves the model after every epoch;
* <code>EarlyStopping</code>, which stops the training when a specified metric of quality has stopped improving;
* <code>ReduceLROnPlateau</code>, which reduces the learning rate when a specified metric of quality has stopped improving; and
* <code>TerminateOnNaN</code>, which terminates the training when a NaN is computed as the loss value.

# Looking at the trained network

We can look at the weights for layer <code>k</code> using <code>model.layers[k].get_weights()</code>.

In [ ]:
print(model.layers[0].get_weights())

## Using TensorBoard

Alternatively, we can use the <code>TensorBoard</code> callback to log information in a format that can be read by the [TensorBoard utility](https://www.tensorflow.org/get_started/summaries_and_tensorboard) in TensorFlow.

# Saving the model

We can use the model's <code>save</code> method to save the model in [HDF5 format](https://support.hdfgroup.org/HDF5).

Saving the model save's the layout and the weights in the neural network, and also the optimizer's state so that you can continue training the model.

In [ ]:
print('saving the model...', end='')
model.save('mnist_cnn.hdf5')
print('done!')

We can load the saved model using <code>keras.models.load_model</code>.

In [ ]:
from keras.models import load_model

model2 = load_model('mnist_cnn.hdf5')

In [ ]:
del model2

# Data augmentation

**Data augmentation** refers to creating new training examples from our training data.  This is especially useful when dealing with image data.

We can create new images by applying random horizontal and vertical shifts to existing images.  This is useful for the digit data since the digits might not be perfectly centered.

Other possible transmogrifications include flipping the image horizontally or vertically, rotating the images, and applying various types of scaling.  These augmentations are not helpful for the digit data.

We will reconstruct the model and train it using data augmentation to increase the effective size of the training set.

In [ ]:
import tensorflow as tf
from keras import models, layers, losses, optimizers

print('reconstructing the model...', end='')

aug = models.Sequential()
aug.add(layers.Conv2D(32, kernel_size=(3, 3),
           activation='relu',
           input_shape=input_shape))
aug.add(layers.Conv2D(64, (3, 3), activation='relu'))
aug.add(layers.MaxPooling2D(pool_size=(2, 2)))
aug.add(layers.Dropout(0.25))
aug.add(layers.Flatten())
aug.add(layers.Dense(128, activation='relu'))
aug.add(layers.Dropout(0.5))
aug.add(layers.Dense(num_classes, activation='softmax'))

aug.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

print('done!')

Now we specify the generator to produce new images.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 1

datagen = ImageDataGenerator(
  width_shift_range=0.1,               # randomly shift images horizontally (fraction of total width)
  height_shift_range=0.1,              # randomly shift images vertically (fraction of total height)
                                       # The remaining options are here for illustrative purposes only.
  featurewise_center=False,            # set input mean to 0 over the dataset
  samplewise_center=False,             # set each sample mean to 0
  featurewise_std_normalization=False, # divide inputs by std of the dataset
  samplewise_std_normalization=False,  # divide each input by its std
  zca_whitening=False,                 # apply ZCA whitening 
  rotation_range=0,                    # randomly rotate images in the range (in degrees, 0 to 180)
  horizontal_flip=False,               # randomly flip images
  vertical_flip=False)                 # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                          steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                          epochs=epochs,
                          validation_data=(x_test, y_test),
                          workers=4)

score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

# A textbook neural network

By a textbook neural network we mean a single layer of units with every input (pixel) connected to every unit.  

The input to each unit is the inner product of the unit's weights with the pixel values plus a bias term.  In Keras we can accomplish this by using a convolution filter the size of the image, and only computing the convolution where the images are perfectly aligned. 

The activation function is the classical sigmoidal function.

In [ ]:
import tensorflow as tf
from keras import models, layers, losses, optimizers

print('constructing the model...', end='')

textbook = models.Sequential()
textbook.add(layers.Conv2D(64, kernel_size=(28,28), padding='valid', 
                           input_shape=input_shape, activation='sigmoid'))
textbook.add(layers.Flatten())
textbook.add(layers.Dense(num_classes, activation='softmax'))

textbook.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

print('done!')

## The model parameters

Let's look at the number of model parameters in our neural network.

This network is considerably simpler than the CNNs we looked.

In [ ]:
textbook.summary()

## Training the network

Because this neural network has fewer parameters than the CNNs, each training epoch goes much faster.

On the other hand, it takes many epochs before we see the accuracy we saw after a single epoch for the more complex models.

In [ ]:
batch_size = 128
epochs = 10

textbook.fit(x_train, y_train_enc,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test_enc))

score = textbook.evaluate(x_test, y_test_enc, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])